In [ ]:
#| echo: false

# Set up:
BLOG_DIR = "blog/posts/spurious-sentience"
using Pkg; Pkg.activate(BLOG_DIR)

# Packages:
using CSV
using DataFrames
using Dates
using Flux
using LinearAlgebra
using Metal
using Plots
using Plots.PlotMeasures
using Printf
using TidierData

# Utility functions:
include(joinpath("$(pwd())", BLOG_DIR, "utils.jl"))

We humans are prone to seek patterns everywhere. Meaningful patterns have often proven to help us make sense of our past, navigate our presence and predict the future. Our society is so invested in finding patterns that today it seems we are more willing than ever to outsource this task to an Artificial Intelligence (AI): an omniscient oracle that leads us down the right path. Unfortunately, history has shown time and again that patterns are double-edged swords: if we attribute the wrong meaning to them, they may lead us nowhere at all, or worse, they may lead us down the dark roads. 

In statistics, misleading patterns are referred to as **spurious relationships**: purely associational relationships between two or more variables that are not causally related to each other at all. The world is full of these and as good as we as species may be at recognizing patterns, we typically have a much harder time discerning spurious relationships from causal ones. Despite new and increased momentum in scientific fields concerned with causal inference and discovery, I am also willing to go out on a limb and claim that we are not about to finally reach the top of Judea Pearl's Causal Ladder through the means of Causal AI.

I agree with the premise that in a world full of spurious relationships, causal reasoning is our only remedy. But I am very sceptical of claims that AI will magically provide that remedy. This leads me to the title and topic of this post: **spurious sentience** - patterns exhibited by artificial intelligence that may hint at sentience but are really just reflections of the data used to train them. The article is written in response to a recent paper and claims by one of the authors, Max Tegmark, that revealed structure in the latent embeddings of Llama 2 should finally have us believe that LLMs are more than just parrots. Since this is an opinionated post, I feel that I should start with a few disclaimers:

1. I take no issue with the methodological ideas that form the foundation of the article in question: on the contrary, I think that mechanistic interpretability is an interesting and important toolkit that can help us better understand the intrinsics and behaviour of opaque artificial intelligences.
2. The visualizations are intriguing, the code is open-sourced and the findings are interesting. 
3. I am surprised that people are surprised by the findings: if we agree that LLMs exhibit strong capabilities that can only be connected to the patterns observed in the data they were trained with, then where exactly do you expect this information to be stored if not in the parameters of the model?^[I would be very surprised---concerned even---if our search for patterns in latent spaces of capable LLMs revealed nothing at all.]
4. I therefore do take issue with the way that these findings are being overblown by people with clout. Perhaps the parrot metaphor should not be taken too literally either, but if anything the paper's findings seem to support the notion that LLMs are remarkably capable of memorizing explicit and implicit knowledge contained in text. 

## Patterns in Latent Spaces and How to Find Them

To demonstrate the claim that observing patterns in latent spaces should not generally surprise us, we will now go through a couple of simple examples. 

### Example: Principal Component Analysis

In response to the claims made by Tegmark, numerous commentators on social media have pointed out that even the simplest of models can exhibit structure in their latent spaces. One of the most popular and illustrative examples I remember from my time at the Bank of England is yield curve decomposition through PCA. The yield curve is a popular tool for investors and economists to gauge the health of the economy. It plots the yields of bonds against their maturities. The slope of the yield curve is often used as a predictor of future economic activity: a steep yield curve is associated with a growing economy, while a flat or inverted yield curve is associated with a contracting economy. 

To understand this better, let us go on a quick detour into economics and look at actual yield curves observed in the US during the Global Financial Crisis (GFC). @fig-gfc-1 shows the yield curve of US Treasury bonds on 27 February 2007, which according to [CNN](https://money.cnn.com/2007/02/27/markets/markets_0630/index.htm?cnn=yes) was a "brutal day on Wall Street".^[The [data](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value_month=202310) is taken from the US Department of the Treasury.] This followed [reports](https://money.cnn.com/2007/02/26/news/economy/greenspan/index.htm?postversion=2007022609) on the previous day of former Federal Reserve Chairman Alan Greenspan's warning that the US economy was at risk of a recession. The yield curve was inverted with a sharp negative spread between the 10-year and 3-month yields, indicative of the market's expectation of a recession.

@fig-gfc-2 shows the corresponding yield curve during the aftermath of the GFC on 20 April 2009. On that day the influential Time Magazine [reported](https://content.time.com/time/business/article/0,8599,1890560,00.html) that the "Banking Crisis is Over". The yield curve was steeply sloped with a positive spread between the 10-year and 3-month yields, indicative of the market's expectation of a recovery. The overall level of the yield curve was still very low though, indicative of the fact that US economy had not fully recovered at that point.


In [ ]:
#| fig-cap: Yield curve of US Treasury bonds.
#| fig-subcap:
#|   - 'Onset of GFC: 27 February 2007.'
#|   - 'Aftermath of GFC: 20 April 2009.'
#| label: fig-gfc
#| output: true
#| layout-ncol: 2

df = CSV.read(joinpath(BLOG_DIR, "data/ust_yields.csv"), DataFrame) |>
    x -> @pivot_longer(x, -Date) |>
    x -> @mutate(x, variable=to_year(variable)) |>
    x -> @mutate(x, year=Dates.year(Date)) |>
    x -> @mutate(x, quarter=Dates.quarter(Date)) |>
    x -> @mutate(x, Date=Dates.format(Date, "yyyy-mm-dd")) |>
    x -> @arrange(x, Date) |>
    x -> @fill_missing(x, "down")
ylims = extrema(skipmissing(df.value))

# Peak-crisis:
onset_date = "2007-02-27"
plt_df = df[df.Date .== onset_date, :]
plt = plot(
    plt_df.variable, plt_df.value;
    label="", color=:blue,
    xlabel="Maturity (years)", ylabel="Yield (%)",
)
scatter!(
    plt_df.variable, plt_df.value;
    label="", color=:blue, alpha=0.5,
    ylims=(0,6)
)
display(plt)

# Post-crisis:
aftermath_date = "2009-04-20"
plt_df = df[df.Date .== aftermath_date, :]
plt = plot(
    plt_df.variable, plt_df.value;
    label="", color=:blue,
    xlabel="Maturity (years)", ylabel="Yield (%)",
)
scatter!(
    plt_df.variable, plt_df.value;
    label="", color=:blue, alpha=0.5,
    ylims=(0,6)
)
display(plt)

Of course, US Treasuries are not the only bonds that are traded in the market. To get a more complete picture of the economy, analysts might therefore be interested in looking at the yield curves of other bonds as well. In particular, we might be interested in predicting economic growth based on the yield curves of many different bonds. The problem with that idea is that it is cursed by high dimensionality: we would end up modelling a single variable of interest (economic growth) with a large number of predictors (the yields of many different bonds). To deal with the curse of high dimensionality it can be useful to decompose the yield curves into sets of principal components. 

To compute the principal components we can decompose the matrix of yields $\mathbf{Y}$ into a product of its singular vectors and values: $\mathbf{Y}=\mathbf{U}\Sigma\mathbf{V}^{\prime}$. I will not go into the details here, because Professor Gilbert Strang has already done a much better job than I ever could in his [Linear Algebra lectures](https://www.youtube.com/watch?v=mBcLRGuAFUk). To put this into the broader context of the article, however, let us simply refer to $\mathbf{U}$, $\Sigma$ and $\mathbf{V}^{\prime}$ as latent embeddings of the yield curve (they are latent because they are not directly observable).

@fig-pca-1 shows the first two principal components of the yield curves of US Treasury bonds over time. Vertical stalks indicate the key dates during the onset and aftermath of the crisis, which we discussed above. For both components, we can observe some marked shifts between the two dates - but can we attribute any meaning to these shifts? It turns out we can: for comparison, @fig-pca-2 shows the average level and spread of the yield curves over time. The first principal component is strongly correlated with the level of the yield curve, while the second principal component is strongly correlated with the spread of the yield curve. To put it in AI-lingo:

> The estimated latent embeddings of the yield curve are characterized by patterns observed in the data. 


In [ ]:
#| fig-cap: Comparison of observed data and latent embeddings of the US Treasury yield curve.
#| fig-subcap:
#|   - Principal components of the yield curve.
#|   - Average level and spread of the yield curve.
#| label: fig-pca
#| output: true

# PCA:
df_wide = @select(df, Date, variable, value) |>
    x -> @pivot_wider(x, names_from = variable, values_from = value) |>
    x -> dropmissing(x)
X = @select(df_wide, -Date) |> Matrix
U, Σ, V = svd(X)
dates = Date.(df_wide.Date)
tick_years = Date.(unique(Dates.year.(dates)))
date_tick = Dates.format.(tick_years, "yyyy")
n_pc = 2
plt_pc = plot(
    dates,
    .-U[:,1:n_pc],
    label=["PC $i" for i in 1:n_pc] |> permutedims,
    size=(1000, 300),
    ylims=(-0.015,0.03),
    legend=:topright
)
plot!(xticks=(tick_years,date_tick), xtickfontsize=6, yaxis=(formatter=y->@sprintf("%.2f",y)))
vline!(Date.([onset_date]), ls=:solid, color=:black, label="Onset of GFC")
vline!(Date.([aftermath_date]), ls=:dash, color=:black, label="Aftermath of GFC")

# Level:
df_level = @group_by(df, Date) |>
    x -> @mutate(x, level=sum(value)/length(value)) |>
    x -> @ungroup(x) |>
    x -> @select(x, Date, level)

# Spreads:
df_spread = @filter(df, variable==0.25 || variable==10) |>
    x -> @select(x, -(year:quarter)) |>
    x -> @mutate(x, variable=ifelse(variable==0.25,"short","long")) |>
    x -> @pivot_wider(x, names_from=variable, values_from=value) |>
    x -> @mutate(x, spread=long-short) |>
    x -> @select(x, Date, spread)

# Plot:
plt_mat = @full_join(df_level, df_spread) |> 
    dropmissing |> 
    unique |>
    x -> @select(x, -Date) |>
    Matrix
plt_obs = plot(
    dates,
    plt_mat,
    label=["Level" "Spread"],
    size=(1000, 300),
    ylims=(-3,9),
    legend=:topright,
    ylab="Yield (%)"
)
plot!(xticks=(tick_years,date_tick), xtickfontsize=6, yaxis=(formatter=y->@sprintf("%.2f",y)))
vline!(Date.([onset_date]), ls=:solid, color=:black, label="Onset of GFC")
vline!(Date.([aftermath_date]), ls=:dash, color=:black, label="Aftermath of GFC")

plot(plt_pc, plt_obs, layout=(2,1), size=(1000, 400), left_margin=5mm, bottom_margin=5mm)

Not convinced? Let us use $\mathbf{Y}=\mathbf{U}\Sigma\mathbf{V}^{\prime}$ in true autoencoder fashion to reconstruct yield curves from principal components. Let $z_1$ denote the first principal component and consider the following: we keep all other $M-1$ principal components fixed at zero where $M$ denotes the total number of maturities; next we traverse the latent space by varying the value of $z_1$ over a fixed grid of length $K$ each time storing the full vector $\mathbf{z}$; finally, we vertically concatenate the vectors and end up with a matrix $\mathbf{Z}$ of dimension $(K \times M)$. To reconstruct yields, we simply multiply $Z$ by the singular values and right singular vectors: $\mathbf{Y}=\mathbf{Z}\Sigma\mathbf{V}^{\prime}$. 

@fig-pca-anim shows the result of this exercise in the left panel. As we can see, our generated yield curves shift vertically as we traverse the latent space. The right panel of @fig-pca-anim shows the result of a similar exercise, but this time we keep the first principal component fixed at zero and vary the second principal component. This time the slope of our generated yield curves shifts as we traverse the latent space. 


In [ ]:
n_vals = 50
pc1_range = range(extrema(U[:,1])..., length=n_vals)
pc2_range = range(extrema(U[:,2])..., length=n_vals)
Z_1 = [[pc1, 0, zeros(size(U, 2)-2)...] for pc1 in pc1_range] |> x -> reduce(vcat, x')
Y_1 = Z_1 * diagm(Σ) * V'
Z_2 = [[0, pc2, zeros(size(U, 2)-2)...] for pc2 in pc2_range] |> x -> reduce(vcat, x')
Y_2 = Z_2 * diagm(Σ) * V'
anim = @animate for i in 1:n_vals
    # Level shifts:
    y = Y_1[i,:]
    p1 = plot(
        unique(df.variable), y;
        label="", color=:blue,
        xlabel="Maturity (years)", ylabel="Yield (%)",
        title="PC1: $(round(collect(pc1_range)[i], digits=5))",
        ylims=extrema(Y_1)
    )
    scatter!(
        unique(df.variable), y;
        label="", color=:blue, alpha=0.5,
    )
    # Spread shifts:
    y = Y_2[i,:]
    p2 = plot(
        unique(df.variable), y;
        label="", color=:blue,
        xlabel="Maturity (years)", ylabel="Yield (%)",
        title="PC2: $(round(collect(pc2_range)[i], digits=5))",
        ylims=extrema(Y_2)
    )
    scatter!(
        unique(df.variable), y;
        label="", color=:blue, alpha=0.5,
    )
    plot(p1, p2, layout=(1,2), size=(1000, 400), left_margin=5mm, bottom_margin=5mm)
end
gif(anim, joinpath(BLOG_DIR, "www/pc_anim.gif"), fps=5)

![Yield curve decomposition through PCA.](www/pc_anim.gif){#fig-pca-anim}

### Example: Deep Learning

So far we have considered simple matrix decomposition. You might argue that principal components are not really latent embeddings in the traditional sense of deep learning. To address this, let us now consider a simple deep-learning example. 

[Data](https://fred.stlouisfed.org/series/GDPC1).


In [ ]:
df_gdp = CSV.read(joinpath(BLOG_DIR, "data/gdp.csv"), DataFrame) |>
    x -> @rename(x, Date=DATE, gdp=GDPC1) |>
    x -> @mutate(x, gdp_l1=lag(gdp)) |>
    x -> @mutate(x, growth=log(gdp)-log(gdp_l1)) |>
    x -> @select(x, Date, growth) |>
    x -> @mutate(x, year=Dates.year(Date)) |>
    x -> @mutate(x, quarter=Dates.quarter(Date)) |>
    x -> @filter(x, year <= 2018)

df_yields_qtr = @group_by(df, year, quarter, variable) |>
    x -> @mutate(x, value=mean(value)) |>
    x -> @ungroup(x) |>
    x -> @select(x, -Date) |>
    unique

df_all = @inner_join(df_gdp, df_yields_qtr, (year, quarter)) |> 
    x -> @pivot_wider(x, names_from=variable, values_from=value) |>
    dropmissing

y = df_all.growth |> 
    x -> Float32.(x)
X = @select(df_all, -(Date:quarter)) |> 
    Matrix |>
    x -> Float32.(x)

# Plot:
p_gdp = plot(
    df_all.Date, y;
    label="", color=:blue,
    size=(800,200),
    ylabel="GDP Growth (%)"
)
p_yields = plot(
    df_all.Date, X;
    label="", color=:blue,
    ylabel="Yield (%)",
    legend=:bottomright,
    alpha=0.5,
    size=(800,400)
)
plot(p_gdp, p_yields, layout=(2,1), size=(800, 600), left_margin=5mm)

In [ ]:
dl = Flux.MLUtils.DataLoader((X', y), batchsize=1, shuffle=true) |> gpu
input_dim = size(X,2)
n_pc = input_dim
n_hidden = 24
encoder = Flux.Chain(
    Dense(input_dim => n_hidden, tanh),
    Dense(n_hidden => n_pc, tanh),
) |> gpu
decoder = Flux.Chain(
    Dense(n_pc => n_hidden, tanh),
    Dense(n_hidden => input_dim, tanh),
) |> gpu
model = Flux.Chain(
    encoder,
    decoder,
    Dense(input_dim, 1),
) |> gpu
loss(yhat, y) = Flux.mse(yhat, y)
opt = Adam()
opt_state = Flux.setup(opt, model)
for epoch in 1:1000
    Flux.train!(model, dl, opt_state) do m, x, y
        loss(m(x), y)
    end
end
yhat = model(X')' |> cpu
plt = plot(yhat, label="Predicted", color=:red)
plot!(y, label="Actual", color=:blue)
display(plt)

# plot(backbone(X')', label="Latent Embeddings")

### Example: Variational Autoencoder



## 🎓 References